In [2]:
import folium
import pandas as pd
import numpy as np

In [18]:
f = folium.Figure()

# center_lat, center_lon = 36.63145, 138.1881
center_lat, center_lon = 80.0, -30.0

m = folium.Map([center_lat, center_lon], zoom_start=5).add_to(f)
folium.Marker(location=[center_lat, center_lon]).add_to(m)
m

In [19]:
N = 19
arr = np.arange(N)

radius = 0.0003

circle_lat = radius * np.sin(arr / N * np.pi * 2) + center_lat
circle_lon = radius * np.cos(arr / N * np.pi * 2) + center_lon

circle_lat_lon = np.stack([circle_lat, circle_lon], axis=1)

df = pd.DataFrame(circle_lat_lon, columns=["lat", "lon"])
df

,lat,lon
0,80.000000,-29.999700
1,80.000097,-29.999716
2,80.000184,-29.999763
3,80.000251,-29.999836
4,80.000291,-29.999926
5,80.000299,-30.000025
6,80.000275,-30.000121
7,80.000221,-30.000203
8,80.000143,-30.000264
9,80.000049,-30.000296


In [20]:
f = folium.Figure()

m = folium.Map([center_lat, center_lon], zoom_start=17).add_to(f)
for i in arr:
    folium.Marker(location=[df["lat"][i], df["lon"][i]]).add_to(m)
m

In [30]:
df.to_csv("csv/circle_sampe_n%d_lat%.4f.csv" % (N, center_lat), index=False)

In [22]:
df_vector = df[1:].copy()
df_vector["lat_prev"] = df[:-1]["lat"].tolist()
df_vector["lon_prev"] = df[:-1]["lon"].tolist()
df_vector["delta_lat"] = df_vector["lat"] - df_vector["lat_prev"]
df_vector["delta_lon"] = df_vector["lon"] - df_vector["lon_prev"]
df_vector["theta"] = np.arctan2(df_vector["delta_lat"], df_vector["delta_lon"])
df_vector

,lat,lon,lat_prev,lon_prev,delta_lat,delta_lon,theta
1,80.000097,-29.999716,80.000000,-29.999700,0.000097,-0.000016,1.736143
2,80.000184,-29.999763,80.000097,-29.999716,0.000087,-0.000047,2.066837
3,80.000251,-29.999836,80.000184,-29.999763,0.000067,-0.000073,2.397531
4,80.000291,-29.999926,80.000251,-29.999836,0.000040,-0.000090,2.728225
5,80.000299,-30.000025,80.000291,-29.999926,0.000008,-0.000098,3.058919
6,80.000275,-30.000121,80.000299,-30.000025,-0.000024,-0.000096,-2.893572
7,80.000221,-30.000203,80.000275,-30.000121,-0.000054,-0.000083,-2.562878
8,80.000143,-30.000264,80.000221,-30.000203,-0.000078,-0.000061,-2.232184
9,80.000049,-30.000296,80.000143,-30.000264,-0.000093,-0.000032,-1.901490
10,79.999951,-30.000296,80.000049,-30.000296,-0.000099,0.000000,-1.570796


In [23]:
def compute_theta_delta(x):
    x = abs(x)
    if (x > 180):
        x = 360 - x
    return x

In [24]:
df_direction = df_vector[1:][["theta"]].copy()
df_direction["theta_prev"] = df_vector["theta"][:-1].tolist()
df_direction["delta_theta"] = np.rad2deg(df_direction["theta"] - df_direction["theta_prev"]).apply(compute_theta_delta)
df_direction

,theta,theta_prev,delta_theta
2,2.066837,1.736143,18.947368
3,2.397531,2.066837,18.947368
4,2.728225,2.397531,18.947368
5,3.058919,2.728225,18.947368
6,-2.893572,3.058919,18.947368
7,-2.562878,-2.893572,18.947368
8,-2.232184,-2.562878,18.947368
9,-1.901490,-2.232184,18.947368
10,-1.570796,-1.901490,18.947368
11,-1.240102,-1.570796,18.947368


In [25]:
df_direction["delta_theta"].sum()

322.10526315792964

In [26]:
def compute_arc_length(lat1, lat2, lon1, lon2):
    """
    緯度経度から距離を計算 (単位球)
    """
    return np.arccos(np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon2 - lon1))

In [27]:
df_3points = df[:-2].copy().rename(columns={"lat": "lat1", "lon": "lon1"})
df_3points["lat2"] = df[1:-1]["lat"].tolist()
df_3points["lon2"] = df[1:-1]["lon"].tolist()
df_3points["lat3"] = df[2:]["lat"].tolist()
df_3points["lon3"] = df[2:]["lon"].tolist()
df_3points = np.deg2rad(df_3points)
c = compute_arc_length(df_3points["lat1"], df_3points["lat2"], df_3points["lon1"], df_3points["lon2"])
b = compute_arc_length(df_3points["lat2"], df_3points["lat3"], df_3points["lon2"], df_3points["lon3"])
a = compute_arc_length(df_3points["lat3"], df_3points["lat1"], df_3points["lon3"], df_3points["lon1"])

df_3points["a"] = a
df_3points["b"] = b
df_3points["c"] = c

df_3points["alpha"] = np.arccos((np.cos(a) - np.cos(b) * np.cos(c)) / (np.sin(b) * np.sin(c)))

# 表示法の変更
pd.options.display.precision = 2

df_3points

,lat1,lon1,lat2,lon2,lat3,lon3,a,b,c,alpha
0,1.4,-0.52,1.4,-0.52,1.4,-0.52,3.22e-06,1.52e-06,1.70e-06,3.07
1,1.4,-0.52,1.4,-0.52,1.4,-0.52,2.71e-06,1.19e-06,1.52e-06,3.05
2,1.4,-0.52,1.4,-0.52,1.4,-0.52,1.92e-06,7.45e-07,1.19e-06,2.95
3,1.4,-0.52,1.4,-0.52,1.4,-0.52,1.01e-06,3.31e-07,7.45e-07,2.39
4,1.4,-0.52,1.4,-0.52,1.4,-0.52,6.52e-07,5.13e-07,3.31e-07,1.73
5,1.4,-0.52,1.4,-0.52,1.4,-0.52,1.47e-06,9.75e-07,5.13e-07,2.80
6,1.4,-0.52,1.4,-0.52,1.4,-0.52,2.34e-06,1.37e-06,9.75e-07,3.02
7,1.4,-0.52,1.4,-0.52,1.4,-0.52,3.00e-06,1.63e-06,1.37e-06,3.07
8,1.4,-0.52,1.4,-0.52,1.4,-0.52,3.36e-06,1.72e-06,1.63e-06,3.08
9,1.4,-0.52,1.4,-0.52,1.4,-0.52,3.36e-06,1.63e-06,1.72e-06,3.08


In [28]:
df_3points["delta_theta"] = 180 - np.rad2deg(df_3points[["alpha"]])
df_3points[["delta_theta"]]

,delta_theta
0,3.82
1,5.42
2,11.05
3,42.91
4,81.07
5,19.47
6,7.22
7,4.35
8,3.42
9,3.38


In [13]:
df_3points["delta_theta"].sum()

329.05253962539405